### OTHMANE NABGOURI

In [96]:
import numpy as np
import pandas as pd
from tabulate import tabulate
import statistics
from statistics import mode

In [245]:
header_list = ["Classe", "X", "Y"]
data_train=pd.read_csv('data_tp1_app.txt', sep = ' ' ,names=header_list)
data_test=pd.read_csv('data_tp1_dec.txt', sep = ' ' ,names=header_list)
data_train2=pd.read_csv('data_tp2_app.txt', sep = ' ' ,names=header_list)
data_test2=pd.read_csv('data_tp2_dec.txt', sep = ' ' ,names=header_list)
data_train3=pd.read_csv('data_tp3_app.txt', sep = ' ' ,names=header_list)
data_test3=pd.read_csv('data_tp3_dec.txt', sep = ' ' ,names=header_list)

### 1-Classifieur 1PPV

In [98]:
def distance_euc(a,b):
        return (a[0]-b[0])**2 + (a[1]-b[1])**2

In [99]:
### Calculons d'abord la distance euclidienne entre chaque echantillon de test et tous les echantillons d'apprentissage
def dist(train,test):
    X= train.shape[0]
    Y= test.shape[0]
    dist = np.zeros((Y,X))
    for i in range (Y):
        for j in range(X):
            dist[i,j]=distance_euc(train.iloc[j,[1,2]],test.iloc[i,[1,2]])
    return dist

In [100]:
def index(dist,test,K):
    "indice des K les plus proches"
    X=test.shape[0]
    indice_deux_proches=np.zeros((X,K))
    for i in range(X):
        indice_deux_proches[i]= pd.Series(dist[i,:]).nsmallest(K).index
    return indice_deux_proches
    
    

In [101]:
### On extrait la classe des echantillons d'apprentissage lesplus proche
def get_class(indice_deux_proches,test,K):
    X=test.shape[0]
    indexes = np.zeros((X,K))
    for k in range(X):
        for j in range(K):
            if indice_deux_proches[k][j] < 100:
                indexes[k][j]=1
            elif indice_deux_proches[k][j] >= 100 and indice_deux_proches[k][j] < 200:
                indexes[k][j]=2
            elif indice_deux_proches[k][j] >= 200 and indice_deux_proches[k][j] < 300:
                indexes[k][j]=3
            elif indice_deux_proches[k][j] >= 300 and indice_deux_proches[k][j] < 400:
                indexes[k][j]=4
            elif indice_deux_proches[k][j] >= 400 and indice_deux_proches[k][j] < 500:
                indexes[k][j]=5
    return indexes

In [103]:
def confusion_matrix(actual, predicted):
    Lenghtt = len(actual)
    top1=0
    # extraire les differents classes
    classes = np.unique(actual)

    matrix = np.zeros((len(classes), len(classes)))

    for i in range(len(classes)):
        for j in range(len(classes)):
            matrix[i, j] = np.sum((actual == classes[i]) & (predicted[:,0] == classes[j]))
    for k in range(Lenghtt):
        if actual[k] == predicted[k,0]:
            top1+=1
    return matrix,top1/Lenghtt

### Test sur les données

On commence par les données du tp1

In [106]:
Dist = dist(data_train,data_test)
indice_deux_proches=index(Dist,data_test,1)
indexes=get_class(indice_deux_proches,data_test,1)
Matrix,top1=confusion_matrix(data_test.Classe,indexes)
Matrix,top1



(array([[ 99.,   0.,   0.,   0.,   1.],
        [  0.,  99.,   0.,   0.,   1.],
        [  0.,   0., 100.,   0.,   0.],
        [  0.,   0.,   0., 100.,   0.],
        [  0.,   0.,   1.,   1.,  98.]]),
 0.992)

Pour les données du tp2

In [42]:
Dist = dist(data_train2,data_test2)
indice_deux_proches=index(Dist,data_test2,1)
indexes=get_class(indice_deux_proches,data_test2,1)
Matrix,top1=confusion_matrix(data_test2.Classe,indexes)
Matrix,top1

(array([[100.,   0.,   0.,   0.,   0.],
        [  0.,  89.,  11.,   0.,   0.],
        [  2.,   4.,  83.,   2.,   9.],
        [  0.,   0.,   3.,  96.,   1.],
        [  0.,   0.,   7.,   1.,  92.]]),
 0.92)

Pour les données du tp3

In [107]:
Dist = dist(data_train3,data_test3)
indice_deux_proches=index(Dist,data_test3,1)
indexes=get_class(indice_deux_proches,data_test3,1)
Matrix,top1=confusion_matrix(data_test3.Classe,indexes)
Matrix,top1

(array([[34., 20., 19., 14., 13.],
        [14., 76.,  4.,  1.,  5.],
        [24.,  4., 71.,  1.,  0.],
        [19.,  2., 12., 65.,  2.],
        [15.,  7.,  1.,  0., 77.]]),
 0.646)

### Conclusion

On remarque qu'avec les données de TP1 et TP2 on a eu de bons performences en utilisant top1 (0.992,0.92) , et avec les données de TP3 , on a eu 0.646 comme taux de classification.

C'est evident car on travaille qu'avec un seul plus proche voisin , donc la performance de notre modéle depend de la distribution des données . Ce qui justifie les résultats ci dessus.

### 2-Kppv avec vote à la majorité et vote à l’unanimité

### Vote à la majorité

L'idée ici est de choisir la classe majoritaire

In [162]:
def majorite(indexes):
    L,k = indexes.shape
    maj = np.zeros(L)
    for i in range(L):
            maj[i]=indexes[i,:].max()
    return maj

In [110]:
def reshape_majorite(indexes_majorite):
    X=indexes_majorite.shape[0]
    maj=indexes_majorite.reshape(X,1)
    return maj

In [151]:
def test(train,test,k):
    Dist = dist(train,test)
    indice_deux_proches=index(Dist,test,k)
    indexes=get_class(indice_deux_proches,test,k)
    major = majorite(indexes)
    redim=reshape_majorite(major)
    Matrix,top1=confusion_matrix(test.Classe,redim)
    return Matrix,top1
    

### Test 

Pour les données TP1

In [152]:
k=5

In [154]:
Matrix,top1=test(data_train,data_test,k)

In [155]:
Matrix,top1

(array([[ 98.,   0.,   0.,   0.,   2.],
        [  0.,  97.,   0.,   0.,   3.],
        [  0.,   0., 100.,   0.,   0.],
        [  0.,   0.,   0., 100.,   0.],
        [  0.,   0.,   0.,   1.,  99.]]),
 0.988)

Pour les données de TP2

In [160]:
Matrix,top1=test(data_train2,data_test2,k)

In [161]:
Matrix,top1

(array([[ 98.,   1.,   1.,   0.,   0.],
        [  0.,  74.,  25.,   1.,   0.],
        [  0.,   2.,  78.,   5.,  15.],
        [  0.,   0.,   3.,  93.,   4.],
        [  0.,   0.,   0.,   0., 100.]]),
 0.886)

Pour les données de TP3

In [158]:
Matrix,top1=test(data_train3,data_test3,k)

In [159]:
Matrix,top1

(array([[ 1., 11., 38., 21., 29.],
        [ 1., 59., 17.,  1., 22.],
        [ 1.,  3., 90.,  5.,  1.],
        [ 0.,  1., 19., 72.,  8.],
        [ 0.,  2.,  2.,  2., 94.]]),
 0.632)

### vote à l’unanimité

Si on trouve que les K echantillons plus proche à l'échantillon de test sont de la meme classe , on prend la classe , sinon on retourne none 

In [178]:
def unaminite(indexes):
    X=indexes.shape[0]
    indice=np.zeros(X)
    for i in range(X):
        if (all(ele == indexes[i][0] for ele in indexes[i,:])) :
            indice[i]=indexes[i][0]
        else :
            indice[i]=None
    return indice
        

In [229]:
def test2(train,test,k):
    Dist = dist(train,test)
    indice_deux_proches=index(Dist,test,k)
    indexes=get_class(indice_deux_proches,test,k)
    unam = unaminite(indexes)
    redim=reshape_majorite(unam)
    Matrix,top1=confusion_matrix(test.Classe,redim)
    return Matrix,top1

### Test

k=5

Pour les données de TP1

In [182]:
Matrix,top1=test2(data_train,data_test,k)

In [183]:
Matrix,top1

(array([[ 98.,   0.,   0.,   0.,   1.],
        [  0.,  97.,   0.,   0.,   0.],
        [  0.,   0., 100.,   0.,   0.],
        [  0.,   0.,   0., 100.,   0.],
        [  0.,   0.,   0.,   0.,  97.]]),
 0.984)

Pour les données de TP2

In [185]:
Matrix,top1=test2(data_train2,data_test2,k)

In [186]:
Matrix,top1

(array([[98.,  0.,  0.,  0.,  0.],
        [ 0., 73.,  0.,  0.,  0.],
        [ 0.,  2., 70.,  1.,  2.],
        [ 0.,  0.,  3., 91.,  0.],
        [ 0.,  0.,  0.,  0., 72.]]),
 0.808)

Pour les données de TP3

In [187]:
Matrix,top1=test2(data_train3,data_test3,k)

In [188]:
Matrix,top1

(array([[ 1.,  3.,  2.,  5.,  7.],
        [ 1., 42.,  0.,  0.,  2.],
        [ 1.,  0., 35.,  1.,  0.],
        [ 0.,  0.,  2., 43.,  0.],
        [ 0.,  1.,  0.,  0., 49.]]),
 0.34)

### 3-Cross Validation

Pour optimiser la valeur de k , on va utiliser uniquement les données d'apprentissage car les données de test sont déstinés uniquement pour le test , et on a pas le droit d'ajuster notre parametre k en les utilisant . Comme ça , on aura un modéle qui est adabté au probleme et non au jeu de données.

on aura 20% des données comme partie test , et 80% comme partie entrainement

In [189]:
def cross_validation(train):
    #On divise en premier temps le jeu de donnée en 5 classes :
    c1 = train[0:100]  
    c2 = train[100:200]
    c3 = train[200:300]
    c4 = train[300:400]
    c5= train[400:500]
    train1= pd.concat([c1.iloc[:80],c2.iloc[:80],c3.iloc[:80],c4.iloc[:80],c5.iloc[:80]],ignore_index=True )
    test1 = pd.concat([c1.iloc[80:100],c2.iloc[80:100],c3.iloc[80:100],c4.iloc[80:100],c5.iloc[80:100]],ignore_index=True )
    
    train2= pd.concat([c1.iloc[20:],c2.iloc[20:],c3.iloc[20:],c4.iloc[20:],c5.iloc[20:]],ignore_index=True )
    test2 = pd.concat([c1.iloc[:20],c2.iloc[:20],c3.iloc[:20],c4.iloc[:20],c5.iloc[:20]],ignore_index=True )
    
    train3= pd.concat([pd.concat([c1.iloc[:20],c1.iloc[40:]]),pd.concat([c2.iloc[:20],c2.iloc[40:]]),pd.concat([c3.iloc[:20],c3.iloc[40:]]),pd.concat([c4.iloc[:20],c4.iloc[40:]]),pd.concat([c5.iloc[:20],c5.iloc[40:]])],ignore_index=True )
    test3 = pd.concat([c1.iloc[20:40],c2.iloc[20:40],c3.iloc[20:40],c4.iloc[20:40],c5.iloc[20:40]],ignore_index=True )

    train4= pd.concat([pd.concat([c1.iloc[:40],c1.iloc[60:]]),pd.concat([c2.iloc[:40],c2.iloc[60:]]),pd.concat([c3.iloc[:40],c3.iloc[60:]]),pd.concat([c4.iloc[:40],c4.iloc[60:]]),pd.concat([c5.iloc[:40],c5.iloc[60:]])],ignore_index=True )
    test4 = pd.concat([c1.iloc[40:60],c2.iloc[40:60],c3.iloc[40:60],c4.iloc[40:60],c5.iloc[40:60]],ignore_index=True )
    
    train5= pd.concat([pd.concat([c1.iloc[:60],c1.iloc[80:]]),pd.concat([c2.iloc[:60],c2.iloc[80:]]),pd.concat([c3.iloc[:60],c3.iloc[80:]]),pd.concat([c4.iloc[:60],c4.iloc[80:]]),pd.concat([c5.iloc[:60],c5.iloc[80:]])],ignore_index=True )
    test5 = pd.concat([c1.iloc[60:80],c2.iloc[60:80],c3.iloc[60:80],c4.iloc[60:80],c5.iloc[60:80]],ignore_index=True )
    
    return train1,test1,train2,test2,train3,test3,train4,test4,train5,test5

### Cherchons la meilleur valeur de K en utilisant le vote à la majorité

### En utilisant les données de TP1

In [220]:
train1,test1,train2,data_test2,train3,donne_test3,train4,test4,train5,test5=cross_validation(data_train)

In [221]:
###Les differntes valeurs de K qu'on va tester
K=[1]
K.extend([k for k in range(5,100,5)] )

In [222]:
def test3(train,DATAtest,list_K):
    for k in list_K:
        Matrix,top1=test(train,DATAtest,k) 
        print("Pour k= " + str(k) + ", on a top1= " + str(top1)  )
    return 0
    

On commence par train1,test1:

In [201]:
test3(train1,test1,K)

Pour k= 1, on a top1= 0.51
Pour k= 5, on a top1= 0.75
Pour k= 10, on a top1= 0.79
Pour k= 15, on a top1= 0.79
Pour k= 20, on a top1= 0.79
Pour k= 25, on a top1= 0.79
Pour k= 30, on a top1= 0.79
Pour k= 35, on a top1= 0.79
Pour k= 40, on a top1= 0.77
Pour k= 45, on a top1= 0.75
Pour k= 50, on a top1= 0.73
Pour k= 55, on a top1= 0.71
Pour k= 60, on a top1= 0.64
Pour k= 65, on a top1= 0.61
Pour k= 70, on a top1= 0.55
Pour k= 75, on a top1= 0.49
Pour k= 80, on a top1= 0.3
Pour k= 85, on a top1= 0.2
Pour k= 90, on a top1= 0.2
Pour k= 95, on a top1= 0.2


0

Pour train2,test2 :

In [223]:
test3(train2,data_test2,K)

Pour k= 1, on a top1= 0.48
Pour k= 5, on a top1= 0.78
Pour k= 10, on a top1= 0.79
Pour k= 15, on a top1= 0.79
Pour k= 20, on a top1= 0.78
Pour k= 25, on a top1= 0.78
Pour k= 30, on a top1= 0.77
Pour k= 35, on a top1= 0.76
Pour k= 40, on a top1= 0.75
Pour k= 45, on a top1= 0.73
Pour k= 50, on a top1= 0.72
Pour k= 55, on a top1= 0.68
Pour k= 60, on a top1= 0.66
Pour k= 65, on a top1= 0.66
Pour k= 70, on a top1= 0.65
Pour k= 75, on a top1= 0.61
Pour k= 80, on a top1= 0.35
Pour k= 85, on a top1= 0.2
Pour k= 90, on a top1= 0.2
Pour k= 95, on a top1= 0.2


0

Pour train3,test3

In [208]:
test3(train3,donne_test3,K)

Pour k= 1, on a top1= 0.5
Pour k= 5, on a top1= 0.74
Pour k= 10, on a top1= 0.79
Pour k= 15, on a top1= 0.78
Pour k= 20, on a top1= 0.78
Pour k= 25, on a top1= 0.77
Pour k= 30, on a top1= 0.77
Pour k= 35, on a top1= 0.77
Pour k= 40, on a top1= 0.75
Pour k= 45, on a top1= 0.74
Pour k= 50, on a top1= 0.72
Pour k= 55, on a top1= 0.71
Pour k= 60, on a top1= 0.66
Pour k= 65, on a top1= 0.63
Pour k= 70, on a top1= 0.59
Pour k= 75, on a top1= 0.53
Pour k= 80, on a top1= 0.38
Pour k= 85, on a top1= 0.2
Pour k= 90, on a top1= 0.2
Pour k= 95, on a top1= 0.2


0

Pour train4, test4

In [204]:
test3(train4,test4,K)

Pour k= 1, on a top1= 0.51
Pour k= 5, on a top1= 0.73
Pour k= 10, on a top1= 0.78
Pour k= 15, on a top1= 0.78
Pour k= 20, on a top1= 0.76
Pour k= 25, on a top1= 0.76
Pour k= 30, on a top1= 0.75
Pour k= 35, on a top1= 0.74
Pour k= 40, on a top1= 0.71
Pour k= 45, on a top1= 0.71
Pour k= 50, on a top1= 0.69
Pour k= 55, on a top1= 0.68
Pour k= 60, on a top1= 0.64
Pour k= 65, on a top1= 0.61
Pour k= 70, on a top1= 0.58
Pour k= 75, on a top1= 0.53
Pour k= 80, on a top1= 0.41
Pour k= 85, on a top1= 0.2
Pour k= 90, on a top1= 0.2
Pour k= 95, on a top1= 0.2


0

Pour train 5,test5

In [206]:
test3(train5,test5,K)

Pour k= 1, on a top1= 0.56
Pour k= 5, on a top1= 0.78
Pour k= 10, on a top1= 0.78
Pour k= 15, on a top1= 0.78
Pour k= 20, on a top1= 0.78
Pour k= 25, on a top1= 0.78
Pour k= 30, on a top1= 0.78
Pour k= 35, on a top1= 0.77
Pour k= 40, on a top1= 0.77
Pour k= 45, on a top1= 0.77
Pour k= 50, on a top1= 0.76
Pour k= 55, on a top1= 0.76
Pour k= 60, on a top1= 0.75
Pour k= 65, on a top1= 0.72
Pour k= 70, on a top1= 0.62
Pour k= 75, on a top1= 0.55
Pour k= 80, on a top1= 0.4
Pour k= 85, on a top1= 0.2
Pour k= 90, on a top1= 0.2
Pour k= 95, on a top1= 0.2


0

### Conclusion

La meilleur valeur de K en utilisant le vote à la majorité égal à 10

On remarque aussi que la performance de notre modéle a diminuer en utilisanr cross validation

### Cherchons la meilleur valeur de K en utilisant le vote à l’unanimité

### En utilisant les données de TP1

In [227]:
train1,test1,train2,data_test2,train3,test3,train4,test4,train5,test5=cross_validation(data_train)

In [228]:
def test_4(train,DATAtest,list_K):
    for k in list_K:
        Matrix,top1=test2(train,DATAtest,k) 
        print("Pour k= " + str(k) + ", on a top1= " + str(top1)  )
    return 0

en utilisant train1,test1

In [230]:
test_4(train1,test1,K)

Pour k= 1, on a top1= 0.51
Pour k= 5, on a top1= 0.23
Pour k= 10, on a top1= 0.21
Pour k= 15, on a top1= 0.2
Pour k= 20, on a top1= 0.2
Pour k= 25, on a top1= 0.2
Pour k= 30, on a top1= 0.2
Pour k= 35, on a top1= 0.2
Pour k= 40, on a top1= 0.18
Pour k= 45, on a top1= 0.18
Pour k= 50, on a top1= 0.18
Pour k= 55, on a top1= 0.18
Pour k= 60, on a top1= 0.17
Pour k= 65, on a top1= 0.16
Pour k= 70, on a top1= 0.15
Pour k= 75, on a top1= 0.12
Pour k= 80, on a top1= 0.03
Pour k= 85, on a top1= 0.0
Pour k= 90, on a top1= 0.0
Pour k= 95, on a top1= 0.0


0

en utilisant train2,test2

In [232]:
test_4(train2,data_test2,K)

Pour k= 1, on a top1= 0.48
Pour k= 5, on a top1= 0.25
Pour k= 10, on a top1= 0.2
Pour k= 15, on a top1= 0.2
Pour k= 20, on a top1= 0.2
Pour k= 25, on a top1= 0.2
Pour k= 30, on a top1= 0.2
Pour k= 35, on a top1= 0.2
Pour k= 40, on a top1= 0.19
Pour k= 45, on a top1= 0.18
Pour k= 50, on a top1= 0.17
Pour k= 55, on a top1= 0.17
Pour k= 60, on a top1= 0.17
Pour k= 65, on a top1= 0.17
Pour k= 70, on a top1= 0.17
Pour k= 75, on a top1= 0.15
Pour k= 80, on a top1= 0.04
Pour k= 85, on a top1= 0.0
Pour k= 90, on a top1= 0.0
Pour k= 95, on a top1= 0.0


0

en utilisant train3,test3

In [235]:
test_4(train3,test3,K)

Pour k= 1, on a top1= 0.5
Pour k= 5, on a top1= 0.25
Pour k= 10, on a top1= 0.22
Pour k= 15, on a top1= 0.2
Pour k= 20, on a top1= 0.2
Pour k= 25, on a top1= 0.19
Pour k= 30, on a top1= 0.19
Pour k= 35, on a top1= 0.19
Pour k= 40, on a top1= 0.17
Pour k= 45, on a top1= 0.17
Pour k= 50, on a top1= 0.16
Pour k= 55, on a top1= 0.16
Pour k= 60, on a top1= 0.15
Pour k= 65, on a top1= 0.15
Pour k= 70, on a top1= 0.14
Pour k= 75, on a top1= 0.13
Pour k= 80, on a top1= 0.05
Pour k= 85, on a top1= 0.0
Pour k= 90, on a top1= 0.0
Pour k= 95, on a top1= 0.0


0

en utilisant train4,test4

In [236]:
test_4(train4,test4,K)

Pour k= 1, on a top1= 0.51
Pour k= 5, on a top1= 0.23
Pour k= 10, on a top1= 0.21
Pour k= 15, on a top1= 0.19
Pour k= 20, on a top1= 0.18
Pour k= 25, on a top1= 0.18
Pour k= 30, on a top1= 0.18
Pour k= 35, on a top1= 0.17
Pour k= 40, on a top1= 0.16
Pour k= 45, on a top1= 0.16
Pour k= 50, on a top1= 0.15
Pour k= 55, on a top1= 0.15
Pour k= 60, on a top1= 0.15
Pour k= 65, on a top1= 0.14
Pour k= 70, on a top1= 0.14
Pour k= 75, on a top1= 0.12
Pour k= 80, on a top1= 0.05
Pour k= 85, on a top1= 0.0
Pour k= 90, on a top1= 0.0
Pour k= 95, on a top1= 0.0


0

en utilisant train5,test5

In [238]:
test_4(train5,test5,K)

Pour k= 1, on a top1= 0.56
Pour k= 5, on a top1= 0.23
Pour k= 10, on a top1= 0.19
Pour k= 15, on a top1= 0.19
Pour k= 20, on a top1= 0.19
Pour k= 25, on a top1= 0.19
Pour k= 30, on a top1= 0.19
Pour k= 35, on a top1= 0.18
Pour k= 40, on a top1= 0.18
Pour k= 45, on a top1= 0.18
Pour k= 50, on a top1= 0.18
Pour k= 55, on a top1= 0.18
Pour k= 60, on a top1= 0.18
Pour k= 65, on a top1= 0.16
Pour k= 70, on a top1= 0.13
Pour k= 75, on a top1= 0.11
Pour k= 80, on a top1= 0.01
Pour k= 85, on a top1= 0.0
Pour k= 90, on a top1= 0.0
Pour k= 95, on a top1= 0.0


0

### Conclusion 

La meilleur valeur de k en utilisant le vote à l'unanimité est 1

### 4-Test avec k=1 en vote à l'unaminité pour les données TP1,TP2,TP3

In [ ]:
data_train=pd.read_csv('data_tp1_app.txt', sep = ' ' ,names=header_list)
data_test=pd.read_csv('data_tp1_dec.txt', sep = ' ' ,names=header_list)
data_train2=pd.read_csv('data_tp2_app.txt', sep = ' ' ,names=header_list)
data_test2=pd.read_csv('data_tp2_dec.txt', sep = ' ' ,names=header_list)
data_train3=pd.read_csv('data_tp3_app.txt', sep = ' ' ,names=header_list)
data_test3=pd.read_csv('data_tp3_dec.txt', sep = ' ' ,names=header_list)

On Commence par TP1

In [240]:
k=1

In [241]:
Matrix,top1=test2(data_train,data_test,k)

In [242]:
Matrix,top1

(array([[ 99.,   0.,   0.,   0.,   1.],
        [  0.,  99.,   0.,   0.,   1.],
        [  0.,   0., 100.,   0.,   0.],
        [  0.,   0.,   0., 100.,   0.],
        [  0.,   0.,   1.,   1.,  98.]]),
 0.992)

In [248]:
###TP2
Matrix,top1=test2(data_train2,data_test2,k)

In [249]:
Matrix,top1

(array([[100.,   0.,   0.,   0.,   0.],
        [  0.,  89.,  11.,   0.,   0.],
        [  2.,   4.,  83.,   2.,   9.],
        [  0.,   0.,   3.,  96.,   1.],
        [  0.,   0.,   7.,   1.,  92.]]),
 0.92)

In [250]:
###TP3
Matrix,top1=test2(data_train3,data_test3,k)

In [251]:
Matrix,top1

(array([[34., 20., 19., 14., 13.],
        [14., 76.,  4.,  1.,  5.],
        [24.,  4., 71.,  1.,  0.],
        [19.,  2., 12., 65.,  2.],
        [15.,  7.,  1.,  0., 77.]]),
 0.646)

### 5-Test avec k=5 en vote à la majorité pour les données TP1,TP2,TP3

In [252]:
k=5

In [254]:
###TP1
Matrix,top1=test(data_train,data_test,k)

In [255]:
Matrix,top1

(array([[ 98.,   0.,   0.,   0.,   2.],
        [  0.,  97.,   0.,   0.,   3.],
        [  0.,   0., 100.,   0.,   0.],
        [  0.,   0.,   0., 100.,   0.],
        [  0.,   0.,   0.,   1.,  99.]]),
 0.988)

In [256]:
###TP2
Matrix,top1=test(data_train2,data_test2,k)

In [257]:
Matrix,top1

(array([[ 98.,   1.,   1.,   0.,   0.],
        [  0.,  74.,  25.,   1.,   0.],
        [  0.,   2.,  78.,   5.,  15.],
        [  0.,   0.,   3.,  93.,   4.],
        [  0.,   0.,   0.,   0., 100.]]),
 0.886)

In [258]:
###TP3
Matrix,top1=test(data_train3,data_test3,k)
Matrix,top1

(array([[ 1., 11., 38., 21., 29.],
        [ 1., 59., 17.,  1., 22.],
        [ 1.,  3., 90.,  5.,  1.],
        [ 0.,  1., 19., 72.,  8.],
        [ 0.,  2.,  2.,  2., 94.]]),
 0.632)